## FAC (SA) PROC and BINDING

---
author: seika<seika@live.ca>   
date: 2025-11-06   

---

基于服务区中心点, 寻找服务区的**表征顶点**(上游最近分流点)

---

Note: 部分异常情况, 通过将 SACP 移出 SA_AREA区域(i.e., 不实用 SA_AREA 携带的 RRL 及其 geom 信息作为 VTX_ST/ED) , 可以通过不相交找到
移出后, CP贴进主线, 这是权宜之计. 未来需要考虑这部分特殊结构的SA的VTX使用手动设置

---

#TODO
- [x] 手动维护更新 SA CP 数据, 提供新开通道路的服务区中心点
- [x] 优化流程, 考虑不使用 AREA 数据, 基于 SA CP 和 融接后的RRL(`gdf_rrl`) 数据 或 路网 DG 获得表征点
- [x] 完善 手动补丁过程
- [x] 简化整体代码

## 🅰 加载数据

In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# Input
PATH_SQLITE_EXT : str = r'/opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib'
PATH_SBDS_ARK : str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/ARK_SBDS.sqlite'
PATH_HDMAP : str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/TEST/HDMAP2505.sqlite'
PATH_CPRN : str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite'

LAYER_NAME_FAC_SA_CP = 'fac_sa_cp_250825'   # Input from SBDS Ark
LAYER_NAME_CPRN_ROADS = 'cprn_roads_cprn_fusionmap_V416_jiangsu'    # Input (A2 Output)

PATH_FOLDER_CPRN_PAYLOAD : str = r'/Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/'     # ✅ 只依赖BASE模型（A2输出）⬇️
MODELNAME_CPRN_BASE : str = 'CPRN_DG_BASE_JS_FUSIONMAP_V416_251028_77387ff4e6adc9c4e2a2e831d15c37cdd7a58a51a160f9660ea0b760b127e3f2.tar.gz'

# Destination Table
LAYER_NAME_CPRN_FACPROJ_SA : str = 'cprn_facproj_sa_dcp_v416_jiangsu'

# 导入自定义模块
from taisl_sop.data.base.spatialite import SpatialitePuller
from taisl_sop.data.sbds.sa_cp import SaCpFetcher
from taisl_sop.model.geohash import Geohash

from loguru import logger

import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

In [2]:
# 获取 SA 中心点
sp_sbds = SpatialitePuller(path_sqlite=PATH_SBDS_ARK, path_spatialite_ext=PATH_SQLITE_EXT,
    set_precision=True, decimal_precision=16, driver='sqlite', verbose=False)
sa_cp_fetcher = SaCpFetcher(
    spatialite_puller = sp_sbds, 
    tb_name = LAYER_NAME_FAC_SA_CP,  # 使用notebook中创建的表名
    verbose = False)

logger.info("\n=== ⟢ 获取所有服务区中心点数据 ⟣ ===")
sa_cp_fetcher.flush_sql()  # 清除之前的SQL
gdf_sa_cp = sa_cp_fetcher.fetch_sa_cp()

gdf_sa_cp = gdf_sa_cp.drop(columns = ['wkt', 'hdmap_bind'])
gdf_sa_cp.set_crs('epsg:4326', inplace=True)


print(f"数据库中总共有 {len(gdf_sa_cp)} 个服务区中心点")

2025-10-29 09:46:04.835 | INFO     | __main__:<module>:9 - 
=== ⟢ 获取所有服务区中心点数据 ⟣ ===


数据库中总共有 246 个服务区中心点


In [ ]:
# 获取 RRL 数据 (融接处理完成, 用于创建CPRN的RRL)

from taisl_sop.data.base.spatialite import SpatialitePuller

sp_cprn = SpatialitePuller(
    path_sqlite=PATH_CPRN, path_spatialite_ext=PATH_SQLITE_EXT,
    set_precision=True, decimal_precision=16, driver='sqlite', verbose=False)

sql_roads = f"""
    SELECT edge_idx, rrl_id, osm_id, ghz_src_vtx, ghz_tgt_vtx, road_code, knd, 
      cls, rtype, mdir, ASText(GEOMETRY) AS wkt
      FROM {LAYER_NAME_CPRN_ROADS};
      """

gdf_roads_cprn = sp_cprn.query_sql_gdf(sql_roads, col_geom=['wkt']).drop(columns=['wkt'])
gdf_roads_cprn.rename_geometry('geom', inplace=True)
gdf_roads_cprn.set_crs('epsg:4326', inplace=True)

print(f"✅ Load `{LAYER_NAME_CPRN_ROADS}` : {len(gdf_roads_cprn)} records")

gdf_roads_cprn.sample(4)

✅ Load `cprn_roads_cprn_fusionmap_V416_jiangsu` : 213795 records


## 🅱 设施投影到路网

In [7]:
gdf_sa_proj = gpd.sjoin_nearest(
    left_df=gdf_sa_cp.to_crs('epsg:4549'),
    right_df=gdf_roads_cprn.to_crs('epsg:4549'),
    how='left', 
    max_distance=500)  # 150m limit)

if gdf_sa_proj['index_right'].isna().sum() > 0:
    logger.warning(f"🚨 有 {gdf_sa_proj['index_right'].isna().sum()} 个 SA_CP 没有找到最近的路网参考线, Will be removed")
    logger.warning(f"{gdf_sa_proj[gdf_sa_proj['index_right'].isna()]}")

gdf_sa_proj = gdf_sa_proj[gdf_sa_proj['index_right'].notna()]
gdf_sa_proj = gdf_sa_proj.rename(columns = {'ghz_src_vtx':'vtx_rrl_st', 'ghz_tgt_vtx':'vtx_rrl_ed'})

2025-10-29 09:46:09.625 | WARNING  | __main__:<module>:8 - 🚨 有 2 个 SA_CP 没有找到最近的路网参考线, Will be removed
2025-10-29 09:46:09.627 | WARNING  | __main__:<module>:9 -            sa_code  sa_fname sa_name  ma_dir roadcode    stake    sa_geohash  \
6  G2503A10000LP0W  龙袍服务区(西)   龙袍服务区     1.0    G2503  K37+000  WTSX5HGF5V17   
9  G2503A10000LP0E  龙袍服务区(东)   龙袍服务区     0.0    G2503  K37+000  WTSX5J5UR992   

                            geom  index_right  edge_idx  rrl_id osm_id  \
6  POINT (402430.018 3563882.15)          NaN       NaN     NaN    NaN   
9  POINT (402447.07 3564074.998)          NaN       NaN     NaN    NaN   

  ghz_src_vtx ghz_tgt_vtx road_code  knd  cls rtype mdir  
6         NaN         NaN       NaN  NaN  NaN   NaN  NaN  
9         NaN         NaN       NaN  NaN  NaN   NaN  NaN  


## 🄲 表征点寻找(从被投影的道路线起讫点进行上下游DC检索)

In [9]:
from cprn.data.pickle import PickleIO

cprn = PickleIO.load_from_pickle(PATH_FOLDER_CPRN_PAYLOAD + MODELNAME_CPRN_BASE, compress = True)
df_sa_rrl = gdf_sa_proj.copy()

File hash checking : 77387ff4e6adc9c4e2a2e831d15c37cdd7a58a51a160f9660ea0b760b127e3f2 : passed


In [10]:
# 对 CPRN_BASE 模型进行分合流点的临时提取与嵌入, 便于后续利用这些分合流点来判定SA的表征点
### EXTRACT DC NODE

import pandas as pd
from taisl_sop.model.topo_nx import TopoNx

dg_cprn = cprn.copy()
dct_dc_nodes = TopoNx.find_divergent_convergent_nodes(dg_cprn)  #TODO: 需要优化

df_dc_nodes = pd.DataFrame(dct_dc_nodes).T.drop(
    columns=['is_divergent', 'is_convergent', 
             'in_edges_attr', 'out_edges_attr', 'node_type'])

def get_dc_type(in_degree, out_degree):
    if in_degree > 1 and out_degree == 1:
        return 'CONV'
    elif in_degree == 1 and out_degree > 1:
        return 'DIV'
    elif in_degree >1 and out_degree > 1:
        return 'MULTI'
    else:
        return 'OTHER'

## make df of dcp fac-like (for further binding)

df_dc_nodes['dc_type'] = df_dc_nodes.apply(
    lambda row: get_dc_type(row['in_degree'], row['out_degree']), axis=1)
df_dc_nodes['gh_dcp'] = df_dc_nodes.index

dct_col_rename_factype = {'CONV': 'DC1', 'DIV': 'DC2', 'MULTI': 'DC3', 'OTHER': 'DCX'}
df_dc_nodes['fac_type'] = df_dc_nodes['dc_type'].map(dct_col_rename_factype)
df_dc_nodes = df_dc_nodes.query("dc_type != 'OTHER'")   # ⚠️ 其他类型不嵌入, e.g., 0->2, 3->0

df_dc_nodes['vtx_fac'] = df_dc_nodes['gh_dcp']
df_dc_nodes['fac_code'] = df_dc_nodes['gh_dcp']
df_dc_nodes['fac_name'] = df_dc_nodes['in_degree'].astype(str) + '|' + df_dc_nodes['out_degree'].astype(str)

from taisl_sop.model.geohash import Geohash
df_dc_nodes['geom_fac_wgs_3d'] = Geohash.decode_df_gdf(df_dc_nodes, colname_geohash='gh_dcp', decode_z=True)['geometry']
df_dc_nodes['geom_fac_wgs'] = Geohash.decode_df_gdf(df_dc_nodes, colname_geohash='gh_dcp', decode_z=False)['geometry']

print(len(df_dc_nodes))

## BIND DCP to CPRN
cols_fac_embd_dcp = ['vtx_fac', 'fac_code', 'fac_type', 'fac_name',]
df_fac_dcp_bind = df_dc_nodes[cols_fac_embd_dcp]

from taisl_sop.model.cprn.fac_embd import FacEmbedder as FacEmbd
dg_cprn = FacEmbd.fac_embed_cprn(cprn = cprn, gdf_fac = df_fac_dcp_bind, 
    col_vtx = 'vtx_fac', inplace = False, verbose = True)

## only for checking embedding result
# from cprn.model.topo.topo_search import CprnTopoSearch as CTS
# CTS.list_fac_df(dg_cprn)

9113


In [11]:
from cprn.model.topo.topo_search import CprnTopoSearch as CTS

def upsearch_dc2(vtx, max_dist = 1000):
    lst_fac_search = CTS.fac_bfs_depth(DG = dg_cprn, start_node = vtx, 
        fac_types = ['DC2'],  direction = 'upstream', 
        max_depth = 1, max_dist = max_dist, mark_max_dist = False,
        query_avoid_edge = None, verbose = False)
    if len(lst_fac_search) > 0:
        return True, lst_fac_search[0]['vtx_intvl_tgt'], lst_fac_search[0]['cumulative_weight'],
    else:
        return False, None, None

def downsearch_dc1(vtx, max_dist = 1000):
    lst_fac_search = CTS.fac_bfs_depth(DG = dg_cprn, start_node = vtx, 
        fac_types = ['DC1'],  direction = 'downstream', 
        max_depth = 1, max_dist = max_dist, mark_max_dist = False,
        query_avoid_edge = None, verbose = False)
    if len(lst_fac_search) > 0:
        return True, lst_fac_search[0]['vtx_intvl_tgt'], lst_fac_search[0]['cumulative_weight'],
    else:
        return False, None, None

In [12]:
# 使用for循环逐一检验 only for manual check
# 本步骤通过后, 可注释跳过, 直接运行后续步骤进行自动表征点提取和持久化

i = 1
for idx, row in df_sa_rrl.iterrows():
    vtx_st = row['vtx_rrl_st']
    dc2_search, vtx_dc2, cum_wgt = upsearch_dc2(vtx_st, max_dist = 1000)
    if dc2_search is False:
        logger.info(f"{i} vtx_st: {vtx_st}, row: {row['sa_fname']}")
        logger.info(f"dc2_search: {dc2_search}, vtx_dc2: {vtx_dc2}, cum_wgt: {cum_wgt}")
    i+= 1

i = 1
for idx, row in df_sa_rrl.iterrows():
    vtx_st = row['vtx_rrl_st']
    dc1_search, vtx_dc1, cum_wgt = downsearch_dc1(vtx_st, max_dist = 1000)
    if dc1_search is False:
        logger.info(f"{i} vtx_st: {vtx_st}, row: {row['sa_fname']}")
        logger.info(f"dc1_search: {dc1_search}, vtx_dc1: {vtx_dc1}, cum_wgt: {cum_wgt}")
    i+= 1

2025-10-29 09:46:13.227 | INFO     | __main__:<module>:9 - 1 vtx_st: WWJ4P2PEY5HK000000, row: 界牌服务区(南)
2025-10-29 09:46:13.228 | INFO     | __main__:<module>:10 - dc2_search: False, vtx_dc2: None, cum_wgt: None
2025-10-29 09:46:13.228 | INFO     | __main__:<module>:9 - 2 vtx_st: WWJ601DVNMVK000000, row: 界牌服务区(北)
2025-10-29 09:46:13.229 | INFO     | __main__:<module>:10 - dc2_search: False, vtx_dc2: None, cum_wgt: None
2025-10-29 09:46:13.235 | INFO     | __main__:<module>:9 - 79 vtx_st: WTTWY3U9YMP8001325, row: 石庄服务区(北)
2025-10-29 09:46:13.236 | INFO     | __main__:<module>:10 - dc2_search: False, vtx_dc2: None, cum_wgt: None
2025-10-29 09:46:13.246 | INFO     | __main__:<module>:9 - 212 vtx_st: WTTR8YDGPF5S001348, row: 泰州大桥服务区(北)
2025-10-29 09:46:13.246 | INFO     | __main__:<module>:10 - dc2_search: False, vtx_dc2: None, cum_wgt: None
2025-10-29 09:46:13.248 | INFO     | __main__:<module>:9 - 235 vtx_st: WTTWY3U3612V001354, row: 石庄服务区(南)
2025-10-29 09:46:13.248 | INFO     | __main__:

In [13]:
df_sa_rrl[['us_div_success', 'us_div_vtx', 'us_weight']] = pd.DataFrame(
    df_sa_rrl['vtx_rrl_st'].apply(lambda x: upsearch_dc2(x, max_dist=1000)).tolist(),
    index=df_sa_rrl.index)

df_sa_rrl[['ds_conv_success', 'ds_conv_vtx', 'ds_weight']] = pd.DataFrame(
    df_sa_rrl['vtx_rrl_st'].apply(lambda x: downsearch_dc1(x, max_dist=1000)).tolist(),
    index=df_sa_rrl.index)

## 🅳 后处理, 
### 🄳1 标准化封装(施绑定所需数据格式)

In [14]:
df_sa_en_bind = df_sa_rrl[df_sa_rrl['us_div_success']][['sa_code', 'sa_fname', 'us_div_vtx', 'sa_geohash']]
df_sa_ex_bind = df_sa_rrl[df_sa_rrl['ds_conv_success']][['sa_code', 'sa_fname', 'ds_conv_vtx', 'sa_geohash']]

df_sa_en_bind = df_sa_en_bind.rename(columns = {'sa_code':'fac_code', 'sa_fname':'fac_name', 'us_div_vtx':'fac_vtx', 'sa_geohash':'fac_geohash'})
df_sa_ex_bind = df_sa_ex_bind.rename(columns = {'sa_code':'fac_code', 'sa_fname':'fac_name', 'ds_conv_vtx':'fac_vtx', 'sa_geohash':'fac_geohash'})

df_sa_en_bind['fac_type'] = 'SA1'
df_sa_ex_bind['fac_type'] = 'SA2'

df_sa_bind = pd.concat([df_sa_en_bind, df_sa_ex_bind], axis = 0)
df_sa_bind = df_sa_bind[['fac_code', 'fac_name', 'fac_type', 'fac_vtx', 'fac_geohash']]
df_sa_bind['fac_ghz'] = df_sa_bind['fac_geohash'] + '-00000'

In [15]:
df_sa_bind.query("fac_code == 'S0035A10TZDQN0S'")

,fac_code,fac_name,fac_type,fac_vtx,fac_geohash,fac_ghz
241,S0035A10TZDQN0S,泰州大桥服务区(南),SA2,WTTPJU4UU3MQ001004,WTTPJGC8ENQJ,WTTPJGC8ENQJ-00000


### 🄳2 手动补丁, 修复自动绑定计算的错误

In [16]:
## ADD Missing SA1/SA2 Bindings
fac_sa_bind_patch = [
    {'fac_code': 'S0035A10TZDQN0S', 'fac_name': '泰大桥服务区(南)', 'fac_type': 'SA1', 
        'fac_vtx': 'WTTPJEW4Q9SB001324', 'fac_geohash': 'WTTPJGC8ENQJ', 'fac_ghz': 'WTTPJGC8ENQJ-00000'},   # ✅ 修复地图错误
    {'fac_code': 'S0018A10000JP0N', 'fac_name': '界牌服务区(北)', 'fac_type': 'SA1', 
        'fac_vtx': 'WWJ601DVNMVK000000', 'fac_geohash': 'WWJ4PCWGXXKF', 'fac_ghz': 'WWJ4PCWGXXKF-00000'},   # ✅ 修复新设服务无拓扑
    {'fac_code': 'S0018A10000JP0N', 'fac_name': '界牌服务区(北)', 'fac_type': 'SA2', 
        'fac_vtx': 'WWJ4P2PSK7ZX000000', 'fac_geohash': 'WWJ4PCWGXXKF', 'fac_ghz': 'WWJ4PCWGXXKF-00000'},   # ✅ 修复新设服务无拓扑
    {'fac_code': 'S0018A10000JP0S', 'fac_name': '界牌服务区(南)', 'fac_type': 'SA1', 
        'fac_vtx': 'WWJ4P2PEY5HK000000', 'fac_geohash': 'WWJ4PCRHVHJD', 'fac_ghz': 'WWJ4PCRHVHJD-00000'},   # ✅ 修复新设服务无拓扑
    {'fac_code': 'S0018A10000JP0S', 'fac_name': '界牌服务区(南)', 'fac_type': 'SA2', 
        'fac_vtx': 'WWJ601DUWCFX000000', 'fac_geohash': 'WWJ4PCRHVHJD', 'fac_ghz': 'WWJ4PCRHVHJD-00000'},   # ✅ 修复新设服务无拓扑
    ]

df_sa_bind_patch = pd.DataFrame(fac_sa_bind_patch)
df_sa_bind = pd.concat([df_sa_bind, df_sa_bind_patch], axis = 0)

## 🌖 fac bind proj line create and serilization

In [17]:
from taisl_sop.model.geohash import Geohash
from shapely.geometry import LineString

df_sa_bind['geom_fac'] = df_sa_bind['fac_ghz'].apply(
    lambda x: Geohash.ghz_decode(x))
df_sa_bind['geom_vtx'] = df_sa_bind['fac_vtx'].apply(
    lambda x: Geohash.ghz_decode(x))

df_sa_bind['geom_fac_proj'] = df_sa_bind.apply(lambda row: LineString([Point(row['geom_fac']), Point(row['geom_vtx'])]), axis=1)
df_sa_bind = df_sa_bind.drop(columns = ['geom_fac', 'geom_vtx'])

gdf_sa_bind = gpd.GeoDataFrame(df_sa_bind, geometry = 'geom_fac_proj').set_crs(epsg=4326)

In [18]:
# 🌕 投影线 Save to Sqlite
print(LAYER_NAME_CPRN_FACPROJ_SA)
gdf_sa_bind.to_file(PATH_CPRN, driver='SQLite', spatialite=True, 
                     layer= LAYER_NAME_CPRN_FACPROJ_SA, overwrite=True,
                     index=False, mode='w', encoding='utf-8')

cprn_facproj_sa_dcp_v416_jiangsu
